In [8]:
import torch
import torchvision
import numpy as np
import random
import matplotlib.pyplot as plt
from torchsummary import summary
import math
from tqdm import tqdm # <- para graficar la barra de avance
import pickle

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# bloque de convolución para emplear en mi red

def conv_block(c_in, c_out, k=5, p=2, s=1, pk=5, ps=1, pp=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s), # conv
        torch.nn.Tanh(),                                      # activation
        torch.nn.MaxPool2d(pk, stride=ps, padding=pp)         # pooling
    )

In [10]:
class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = conv_block(n_channels, 16)
    self.conv1_out = None
    self.conv2 = conv_block(16, 8)
    self.conv2_out = None
    self.conv3 = conv_block(8, 4)
    self.conv3_out = None
    self.conv4 = conv_block(4, 4)
    self.conv4_out = None
    self.conv5 = conv_block(4, 4)
    self.conv5_out = None
    self.conv6 = conv_block(4, 4)
    self.conv6_out = None
    self.conv7 = conv_block(4, 4)
    self.conv7_out = None
    self.conv8 = conv_block(4, 4)
    self.conv8_out = None
    self.conv9 = conv_block(4, 4)
    self.conv9_out = None
    self.conv10 = conv_block(4, 4)
    self.conv10_out = None
    self.conv11 = conv_block(4, 4)
    self.conv11_out = None
    self.conv12 = conv_block(4, 4)
    self.conv12_out = None
    self.conv13 = conv_block(4, 4)
    self.conv13_out = None
    self.fc = torch.nn.Linear(3136, n_outputs) # verificar la dim de la salida para calcular el tamaño de la fully conected!!
    
    
    print('Red creada')
    print('arquitectura:')
    print(self)
    
    # Me fijo en el número de capas
    i=0
    for layer in self.children():
        i=i+1
    print('Número total de capas de CNN (conv+act+polling) + finales : ', i)
    
    # Me fijo en el número de parámetros entrenables
    pytorch_total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
    print('Número total de parámetros a entrenar: ', pytorch_total_params)

  def n_layers(self):
    i=0
    for layer in self.children():
        i=i+1

    return i

  def n_parameters(self):
    pytorch_total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
    return pytorch_total_params

  def validar_dim(self, tam):
    print("Validacion de dimensiones")
    x = torch.randn(1, 1, int(tam), int(tam))
    print("Tamaño entrada: ", x.shape)
    x = self.conv1(x)
    print("Tamaño salida conv1: ", x.shape)
    x = self.conv2(x)
    print("Tamaño salida conv2: ", x.shape)

    x = self.conv3(x)
    print("Tamaño salida conv3: ", x.shape)

    x = self.conv4(x)
    print("Tamaño salida conv4: ", x.shape)

    x = self.conv5(x)
    print("Tamaño salida conv5: ", x.shape)

    x = self.conv6(x)
    print("Tamaño salida conv6: ", x.shape)

    x = self.conv7(x)
    print("Tamaño salida conv7: ", x.shape)

    x = self.conv8(x)
    print("Tamaño salida conv8: ", x.shape)

    x = self.conv9(x)
    print("Tamaño salida conv9: ", x.shape)

    x = self.conv10(x)
    print("Tamaño salida conv10: ", x.shape)

    x = self.conv11(x)
    print("Tamaño salida conv11: ", x.shape)

    x = self.conv12(x)
    print("Tamaño salida conv12: ", x.shape)

    x = self.conv13(x)
    print("Tamaño salida conv13: ", x.shape)

    x = x.view(x.shape[0], -1)
    print("Tamaño imagen vectorizada: ", x.shape)
    x = self.fc(x)
    print("Tamaño salida fc (nro clases): ", x.shape)

  def forward(self, x):
    self.conv1_out = self.conv1(x)
    self.conv2_out = self.conv2(self.conv1_out)
    self.conv3_out = self.conv3(self.conv2_out)
    self.conv4_out = self.conv4(self.conv3_out)
    self.conv5_out = self.conv5(self.conv4_out)
    self.conv6_out = self.conv6(self.conv5_out)
    self.conv7_out = self.conv7(self.conv6_out)
    self.conv8_out = self.conv8(self.conv7_out)
    self.conv9_out = self.conv9(self.conv8_out)
    self.conv10_out = self.conv10(self.conv9_out)
    self.conv11_out = self.conv11(self.conv10_out)
    self.conv12_out = self.conv12(self.conv11_out)
    self.conv13_out = self.conv13(self.conv12_out)
    # self.conv4_out = self.conv4(self.conv3_out)
    y = self.conv13_out.view(self.conv13_out.shape[0], -1)
    y = self.fc(y)
    # x = self.sm(x)
    return y
    

In [6]:
def fit(model, dataloader, epochs=15):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss()
    epoch_t_loss = []
    epoch_v_loss = []
    epoch_t_acc = []
    epoch_v_acc = []
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            X = X.unsqueeze(0)
            X = X.permute(1, 0, 2, 3)
            y_hat = model(X)
            loss = criterion(y_hat, y.type(torch.LongTensor))
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            ####
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        epoch_t_loss.append(np.mean(train_loss))
        epoch_t_acc.append(np.mean(train_acc))
        
        
    plt.figure()
    plt.plot(epoch_t_loss, 'r')
    plt.title('loss')
    plt.legend(['train loss'])
    plt.grid()

    plt.figure()
    plt.plot(epoch_t_acc, 'r')
    plt.title('acc')
    plt.legend(['train acc'])
    plt.grid()
    plt.show()

In [7]:
def test_acc(model, dataloader):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss()
    train_acc = []

    bar = tqdm(dataloader['test'])
    for batch in bar:
        X, y = batch
        X = X.unsqueeze(0)
        X = X.permute(1, 0, 2, 3)
        X, y = X.to(device), y.to(device)
        y_hat = model(X)
        acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
        train_acc.append(acc)
        bar.set_description(f"acc {np.mean(train_acc):.5f}")
        
    v_acc  = np.mean(train_acc)

    print(f"Test accuracy {np.mean(v_acc):.5f}")
    plt.figure()
    plt.plot(train_acc, 'r')


    return v_acc